In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import re

In [2]:
date_cols = {
    'olist_orders_dataset.csv': [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date',
    ],
    'olist_order_items_dataset.csv': [
        'shipping_limit_date',
    ],
    'olist_order_reviews_dataset.csv': [
        'review_creation_date',
        'review_answer_timestamp',
    ],
    # The following datasets have NO date columns:
    # 'olist_customers_dataset.csv'
    # 'olist_geolocation_dataset.csv'
    # 'olist_order_payments_dataset.csv'
    # 'olist_products_dataset.csv'
    # 'olist_sellers_dataset.csv'
    # 'product_category_name_translation.csv'
}

def read_olist_csv(path):
    """
    Reads an Olist CSV and parses dates for the correct columns.
    Args:
        path (str): Path to the CSV file.
    Returns:
        pd.DataFrame: Loaded dataframe with date columns parsed as datetime.
    """
    # Extract just the filename, e.g., 'olist_orders_dataset.csv'
    filename = os.path.basename(path)
    # Get the correct date columns for this file, or an empty list
    parse_dates = date_cols.get(filename, [])
    # Read the CSV, parsing the specified date columns (if any)
    return pd.read_csv(path, parse_dates=parse_dates)

df_orders      = read_olist_csv('../data/olist_orders_dataset.csv')
df_customers   = read_olist_csv('../data/olist_customers_dataset.csv')
df_items       = read_olist_csv('../data/olist_order_items_dataset.csv')
df_payments    = read_olist_csv('../data/olist_order_payments_dataset.csv')
df_reviews     = read_olist_csv('../data/olist_order_reviews_dataset.csv')
df_products    = read_olist_csv('../data/olist_products_dataset.csv')
df_prod_cat_tr = read_olist_csv('../data/product_category_name_translation.csv')
df_sellers     = read_olist_csv('../data/olist_sellers_dataset.csv')
df_geo         = read_olist_csv('../data/olist_geolocation_dataset.csv')

In [3]:
# Load the orders dataset:
df_orders = read_olist_csv("../data/olist_orders_dataset.csv")
df_orders.head(10)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07
8,76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23 18:29:09,2017-01-25 02:50:47,2017-01-26 14:16:31,2017-02-02 14:08:10,2017-03-06
9,e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29 11:55:02,2017-07-29 12:05:32,2017-08-10 19:45:24,2017-08-16 17:14:30,2017-08-23


| Column Name                    | Description                                                                 |
|-------------------------------|-----------------------------------------------------------------------------|
| `order_id`                    | Unique identifier for each order.                                           |
| `customer_id`                 | Unique identifier for the customer who placed the order.                    |
| `order_status`                | Current status of the order (e.g., delivered, shipped, etc.).     |
| `order_purchase_timestamp`    | Date and time when the order was placed.                                    |
| `order_approved_at`           | Date and time when the order was approved for processing.                   |
| `order_delivered_carrier_date`| Date the order was handed over to the logistics/delivery carrier.           |
| `order_delivered_customer_date`| Date the order was delivered to the customer.                              |
| `order_estimated_delivery_date`| Estimated delivery date promised to the customer at the time of purchase.  |


In [4]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [5]:
df_orders.describe()

,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99281,97658,96476,99441
mean,2017-12-31 08:43:12.776581120,2017-12-31 18:35:24.098800128,2018-01-04 21:49:48.138278656,2018-01-14 12:09:19.035542272,2018-01-24 03:08:37.730111232
min,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00
25%,2017-09-12 14:46:19,2017-09-12 23:24:16,2017-09-15 22:28:50.249999872,2017-09-25 22:07:22.249999872,2017-10-03 00:00:00
50%,2018-01-18 23:04:36,2018-01-19 11:36:13,2018-01-24 16:10:58,2018-02-02 19:28:10.500000,2018-02-15 00:00:00
75%,2018-05-04 15:42:16,2018-05-04 20:35:10,2018-05-08 13:37:45,2018-05-15 22:48:52.249999872,2018-05-25 00:00:00
max,2018-10-17 17:30:18,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00


In [6]:
summary = []

for col in df_orders.columns:
    unique_vals = df_orders[col].dropna().unique()
    summary.append({
        'Column': col,
        'Unique Count': len(unique_vals),
        'Unique Values': unique_vals
    })

df_summary = pd.DataFrame(summary)
df_summary

,Column,Unique Count,Unique Values
0,order_id,99441,"[e481f51cbdc54678b7cc49136f2d6af7, 53cdb2fc8bc..."
1,customer_id,99441,"[9ef432eb6251297304e76186b10a928d, b0830fb4747..."
2,order_status,8,"[delivered, invoiced, shipped, processing, una..."
3,order_purchase_timestamp,98875,"[2017-10-02 10:56:33, 2018-07-24 20:41:37, 201..."
4,order_approved_at,90733,"[2017-10-02 11:07:15, 2018-07-26 03:24:27, 201..."
5,order_delivered_carrier_date,81018,"[2017-10-04 19:55:00, 2018-07-26 14:31:00, 201..."
6,order_delivered_customer_date,95664,"[2017-10-10 21:25:13, 2018-08-07 15:27:45, 201..."
7,order_estimated_delivery_date,459,"[2017-10-18 00:00:00, 2018-08-13 00:00:00, 201..."


In [25]:
# Unique order_status values:
print(df_orders['order_status'].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [26]:
# Get a sample row for each order_status value
status_examples = df_orders.groupby('order_status').first().reset_index()

# Create a display-friendly dataframe
display_df = pd.DataFrame()

for _, row in status_examples.iterrows():
    # Format the row data
    formatted_row = row.copy()
    
    # Format datetime columns to be more readable
    for col in row.index:
        if pd.api.types.is_datetime64_any_dtype(row[col]):
            formatted_row[col] = "N/A" if pd.isna(row[col]) else row[col].strftime('%Y-%m-%d %H:%M:%S')
    
    # Add the status name as a separate row for better visualization
    display_df = pd.concat([display_df, pd.DataFrame([formatted_row])])

# Display the dataframe with better formatting
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Wider display
pd.set_option('display.colheader_justify', 'center')  # Center column headers

# create a more compact display focusing on key columns
compact_view = display_df[['order_status', 'order_purchase_timestamp', 'order_approved_at', 
                         'order_delivered_carrier_date', 'order_delivered_customer_date', 
                         'order_estimated_delivery_date']]

print("\n=== COMPACT ORDER STATUS EXAMPLES ===\n")
display(compact_view)


=== COMPACT ORDER STATUS EXAMPLES ===



,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,approved,2017-02-06 20:18:17,2017-02-06 20:30:19,NaT,NaT,2017-03-01
1,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,2018-01-29 22:33:25,2018-03-21 22:03:51,2018-08-14
2,created,2017-12-05 01:07:52,NaT,NaT,NaT,2018-01-11
3,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
4,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
5,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03
6,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28
7,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaT,NaT,2017-12-05


For our project scope, we are only interested in "Delivered" orders since we are looking to evaluate the delivery performance

In [27]:
# Keep only delivered orders:
df_orders_delivered = df_orders[df_orders['order_status'] == 'delivered'].copy()
df_orders_delivered.shape

(96478, 8)

In [28]:
df_orders_delivered.duplicated().sum()

0

In [35]:
df_orders_delivered.drop(columns='order_id').duplicated().sum()

0

In [29]:
df_orders_delivered.isna().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
dtype: int64

Missing columns:

|order_approved_at|                 160|

|order_delivered_carrier_date|     1783|

|order_delivered_customer_date|    2965|

# Data Cleaning

## Handling Missing Values

### Missing `order_approved_at`

In [8]:
# Filter rows where 'order_approved_at' is missing:
missing_approved = df_orders[df_orders['order_approved_at'].isna()]

# Display the rows:
print(missing_approved[['order_status', 'order_purchase_timestamp', 'order_approved_at',
                        'order_delivered_carrier_date', 'order_delivered_customer_date',
                        'order_estimated_delivery_date']].head(10))


     order_status order_purchase_timestamp order_approved_at  \
1130     canceled      2018-08-28 15:26:39               NaT   
1801     canceled      2018-09-20 13:54:16               NaT   
1868     canceled      2017-03-04 12:14:30               NaT   
2029     canceled      2018-08-29 16:27:49               NaT   
2161     canceled      2017-05-01 16:12:39               NaT   
3056     canceled      2016-10-04 19:41:32               NaT   
3094     canceled      2018-08-21 10:25:18               NaT   
3684     canceled      2018-08-07 16:33:59               NaT   
4373     canceled      2018-08-09 14:54:47               NaT   
4939     canceled      2018-08-22 18:52:29               NaT   

     order_delivered_carrier_date order_delivered_customer_date  \
1130                          NaT                           NaT   
1801                          NaT                           NaT   
1868                          NaT                           NaT   
2029                       

In [14]:
# Unique order_status values where order_approved_at is missing:
print(missing_approved['order_status'].unique())

['canceled' 'delivered' 'created']


In [21]:
# Delivered orders with missing order_approved_at:
delivered_missing = missing_approved[missing_approved['order_status'] == 'delivered']
display(delivered_missing)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaT,2017-02-23 12:04:47,2017-03-01 13:25:33,2017-03-17
16567,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,NaT,2017-02-23 09:01:52,2017-03-02 10:05:06,2017-03-21
19031,7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:47,NaT,2017-02-22 16:25:25,2017-03-01 08:07:38,2017-03-17
22663,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:35,NaT,2017-02-22 11:23:10,2017-03-09 07:28:47,2017-03-31
23156,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,NaT,2017-02-22 11:23:11,2017-03-02 11:09:19,2017-03-20
26800,c1d4211b3dae76144deccd6c74144a88,684cb238dc5b5d6366244e0e0776b450,delivered,2017-01-19 12:48:08,NaT,2017-01-25 14:56:50,2017-01-30 18:16:01,2017-03-01
38290,d69e5d356402adc8cf17e08b5033acfb,68d081753ad4fe22fc4d410a9eb1ca01,delivered,2017-02-19 01:28:47,NaT,2017-02-23 03:11:48,2017-03-02 03:41:58,2017-03-27
39334,d77031d6a3c8a52f019764e68f211c69,0bf35cac6cc7327065da879e2d90fae8,delivered,2017-02-18 11:04:19,NaT,2017-02-23 07:23:36,2017-03-02 16:15:23,2017-03-22
48401,7002a78c79c519ac54022d4f8a65e6e8,d5de688c321096d15508faae67a27051,delivered,2017-01-19 22:26:59,NaT,2017-01-27 11:08:05,2017-02-06 14:22:19,2017-03-16
61743,2eecb0d85f281280f79fa00f9cec1a95,a3d3c38e58b9d2dfb9207cab690b6310,delivered,2017-02-17 17:21:55,NaT,2017-02-22 11:42:51,2017-03-03 12:16:03,2017-03-20


These rows show orders marked as `'delivered'` with missing `order_approved_at` timestamps. This breaks the expected flow — purchase → approval → delivery. Possible reasons include:
- Approval event was not properly recorded.
- Snapshot timing issue—approval data not yet flushed when extracted.
- Legacy or early 2017 data may have incomplete approval tracking.

Because these orders are only missing `order_approved_at`, and the other delivery-related dates are normal, we will keep these rows and replace the missing values with X

### Missing `order_delivered_customer_date`

In [31]:
# Delivered orders but missing order_delivered_customer_date:
missing_delivery = df_orders_delivered[df_orders_delivered['order_delivered_customer_date'].isna()]
display(missing_delivery)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,2017-11-28 17:56:40,2017-11-30 18:12:23,NaT,2017-12-18
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,2018-06-20 07:19:05,2018-06-25 08:05:00,NaT,2018-07-16
43834,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,2018-07-01 17:15:12,2018-07-03 13:57:00,NaT,2018-07-30
79263,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,2018-07-01 22:15:14,2018-07-03 13:57:00,NaT,2018-07-30
82868,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,2018-07-01 21:29:54,2018-07-03 09:28:00,NaT,2018-07-24
92643,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaT,NaT,2017-06-23
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,2018-06-08 12:36:39,2018-06-12 14:10:00,NaT,2018-06-26
98038,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,2018-06-27 16:29:30,2018-07-03 19:26:00,NaT,2018-07-19


Handling Missing order_delivered_customer_date in Delivered Orders

Found 8 orders with status "delivered" but missing the actual delivery date.

These records cannot be used to calculate delivery performance or lateness, so they were dropped from the analysis.

Impact: This step ensures that only complete, analyzable delivery events are used, avoiding bias or errors in our delivery metrics.

We did not impute missing delivered dates because there is no reliable, business-justified way to infer them. For operational KPIs, only factual delivery events are included. Since it’s less than 0.01% of orders (8 out of 96k rows), dropping them protects metric accuracy and business trust, following industry best practice

In [32]:
# Drop rows with missing order_delivered_customer_date:
df_orders_delivered = df_orders_delivered[df_orders_delivered['order_delivered_customer_date'].notna()]

### Missing `order_delivered_carrier_date`

In [33]:
# Delivered orders but missing order_delivered_carrier_date:
missing_carrier = df_orders_delivered[df_orders_delivered['order_delivered_carrier_date'].isna()]
display(missing_carrier)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
73222,2aa91108853cecb43c84a5dc5b277475,afeb16c7f46396c0ed54acb45ccaaa40,delivered,2017-09-29 08:52:58,2017-09-29 09:07:16,NaT,2017-11-20 19:44:47,2017-11-14


For delivery performance from the customer perspective, this order is valid, we know when it was delivered. We can't remove this row since it provides valuable example of a delivery performance, especially since this particular order was delivered late.

However, for seller-specific lateness KPIs (e.g. how often this seller is late to deliver the items to Olist carrier), we can’t fairly evaluate this seller, because we have no record of when they handed the order off. Imputing that value would introduce unjustified assumptions and risk biasing our seller performance insights.

So **we keep this row** for customer delivery metrics, but exclude it from seller-side performance analysis **without doing any imputation**.

In [34]:
df_orders_delivered.isna().sum()

order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      1
order_delivered_customer_date     0
order_estimated_delivery_date     0
dtype: int64

## Handling Illogical Rows

| Edge Case                                                   | Why It Matters                                      | How to Detect / Handle                                                                                                                          |
| ----------------------------------------------------------- | --------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------- |
| **Delivered before approved**   done                            | Negative “handoff” period; nonsense.                | `mask = df['order_delivered_carrier_date'] < df['order_approved_at']`<br>Inspect these rows; drop or correct if you can trace a data entry bug. |
| **Approved before purchase**  done                              | Approval shouldn’t precede purchase.                | `mask = df['order_approved_at'] < df['order_purchase_timestamp']`<br>Usually safe to drop—very likely ETL error.                                |
| **Carrier date before purchase**                            | Seller can’t hand off before sale.                  | `mask = df['order_delivered_carrier_date'] < df['order_purchase_timestamp']`                                                                    |
| **Customer delivery before carrier** done                       | Delivery can’t happen before handoff.               | `mask = df['order_delivered_customer_date'] < df['order_delivered_carrier_date']`                                                               |
| **Estimated date before purchase**                          | Bad forecast logic or data error.                   | `mask = df['order_estimated_delivery_date'] < df['order_purchase_timestamp']`                                                                   |

In [40]:
df_orders_delivered

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [63]:
# List out the “should-be-before” relationships
date_pairs = [
    ('order_purchase_timestamp',     'order_approved_at'),
    ('order_purchase_timestamp',     'order_delivered_carrier_date'),
    ('order_purchase_timestamp',     'order_estimated_delivery_date'),
    ('order_purchase_timestamp',     'order_delivered_customer_date'),
    ('order_approved_at',            'order_delivered_carrier_date'),
    ('order_approved_at',            'order_estimated_delivery_date'),
    ('order_approved_at',            'order_delivered_customer_date'),
    ('order_delivered_carrier_date', 'order_delivered_customer_date'),
]

In [64]:
anomaly_masks = {}

for earlier, later in date_pairs:
    mask_name = f"{earlier}_before_{later}"
    anomaly_masks[mask_name] = df_orders_delivered[later] < df_orders_delivered[earlier]

In [65]:
# Combine all the masks with OR
combined_mask = pd.concat(anomaly_masks.values(), axis=1).any(axis=1)

# Show you which rule(s) each bad row breaks
violations = pd.DataFrame(anomaly_masks).loc[combined_mask]
violations['order_id'] = df_orders_delivered.loc[combined_mask, 'order_id'].values

display(violations)

,order_purchase_timestamp_before_order_approved_at,order_purchase_timestamp_before_order_delivered_carrier_date,order_purchase_timestamp_before_order_estimated_delivery_date,order_purchase_timestamp_before_order_delivered_customer_date,order_approved_at_before_order_delivered_carrier_date,order_approved_at_before_order_estimated_delivery_date,order_approved_at_before_order_delivered_customer_date,order_delivered_carrier_date_before_order_delivered_customer_date,order_id
15,False,False,False,False,True,False,False,False,dcb36b511fcac050b97cd5c05de84dc3
64,False,False,False,False,True,False,False,False,688052146432ef8253587b930b01a06d
199,False,False,False,False,True,False,True,False,58d4c4747ee059eeeb865b349b41f53a
210,False,False,False,False,True,False,False,False,412fccb2b44a99b36714bca3fef8ad7b
415,False,False,False,False,True,False,False,False,56a4ac10a4a8f2ba7693523bb439eede
...,...,...,...,...,...,...,...,...,...
99091,False,False,False,False,True,False,False,False,240ead1a7284667e0ec71d01f80e4d5e
99230,False,True,False,False,True,False,False,False,78008d03bd8ef7fcf1568728b316553c
99266,False,False,False,False,True,False,False,False,76a948cd55bf22799753720d4545dd2d
99377,False,False,False,False,True,False,False,False,a6bd1f93b7ff72cc348ca07f38ec4bee


"True" here means that the rule is broken. If an order don't have a temporal inconsistency, then it should have a "False" value on every rule column here

In [66]:
violations.drop(columns='order_id').value_counts()

order_purchase_timestamp_before_order_approved_at  order_purchase_timestamp_before_order_delivered_carrier_date  order_purchase_timestamp_before_order_estimated_delivery_date  order_purchase_timestamp_before_order_delivered_customer_date  order_approved_at_before_order_delivered_carrier_date  order_approved_at_before_order_estimated_delivery_date  order_approved_at_before_order_delivered_customer_date  order_delivered_carrier_date_before_order_delivered_customer_date
False                                              False                                                         False                                                          False                                                          True                                                   False                                                   False                                                   False                                                                1124
                                                

#### Temporal Consistency Checks & Anomaly Interpretation

To ensure reliable and logical delivery performance metrics, we ran a series of temporal consistency checks on all key timestamp columns in the `orders` dataset. These checks validate that each stage in the order lifecycle follows a logical chronological order, such as:

* **Purchase happens before approval**
* **Approval before seller handoff**
* **Seller handoff before customer delivery**
* **Order events occur before estimated delivery date**

#### **Findings**

Our analysis revealed several types of anomalies, summarized below:

* **1124 orders:** Handoff to Olist (carrier date) is recorded **before** the order was even approved.
  *→ This is impossible, as Olist must approve the order before the seller can hand it off.*
* **165 orders:** Handoff date is **before both purchase and approval dates**.
  *→ This is even less plausible, as the seller cannot ship an order that has not been placed or approved.*
* **56 orders:** Both the handoff and final delivery dates occur **before order approval**.
  *→ Again, this breaks the required sequence of events.*
* **23 orders:** The package is shown as delivered to the customer **before it was handed to the carrier**.
  *→ The product cannot be received by the customer before it has left the seller.*
* **5 orders:** Estimated delivery date is **before the order was even approved**.
  *→ This likely reflects a forecasting or data entry error.*
* **1 order:** Multiple stages (handoff, estimated delivery, final delivery) occur **before order approval**.
  *→ These are extreme anomalies indicating deeper data integrity issues.*

In total, these anomalies represent about **1.4% of delivered orders**. Such issues almost certainly result from data entry mistakes or ETL errors and could severely bias our delivery performance analysis if left unaddressed.

In [67]:
import numpy as np

# Remove the order_id column to focus on anomalies only
anomaly_cols = [col for col in violations.columns if col != 'order_id']

# Find all unique anomaly combinations (as boolean tuples)
unique_anomaly_cases = violations[anomaly_cols].drop_duplicates()

# Create a dict of DataFrames: each combination -> DataFrame of those rows
anomaly_case_dfs = {}
for idx, unique_case in unique_anomaly_cases.iterrows():
    mask = np.all(violations[anomaly_cols].values == unique_case.values, axis=1)
    # OR, with DataFrame logic:
    # mask = (violations[anomaly_cols] == unique_case.values).all(axis=1)
    key = tuple(unique_case.values)
    anomaly_case_dfs[key] = violations.loc[mask].copy()

# Example: Show all keys
for k, df_case in anomaly_case_dfs.items():
    print(f"Anomaly case {k}: {df_case.shape[0]} rows")
    # display(df_case.head())  # uncomment to view sample rows

Anomaly case (False, False, False, False, True, False, False, False): 1124 rows
Anomaly case (False, False, False, False, True, False, True, False): 56 rows
Anomaly case (False, True, False, False, True, False, False, False): 165 rows
Anomaly case (False, False, False, False, False, False, False, True): 23 rows
Anomaly case (False, False, False, False, True, True, True, False): 5 rows
Anomaly case (False, False, False, False, False, True, False, False): 1 rows


### Actions Taken

#### First Case: Handoff to Olist before Order Approval

**1132 orders:** Handoff to Olist (carrier date) is recorded **before** the order was even approved. Example:

In [68]:
first_case = list(anomaly_case_dfs.keys())[0]
df_first_case = anomaly_case_dfs[first_case]
display(df_first_case)

,order_purchase_timestamp_before_order_approved_at,order_purchase_timestamp_before_order_delivered_carrier_date,order_purchase_timestamp_before_order_estimated_delivery_date,order_purchase_timestamp_before_order_delivered_customer_date,order_approved_at_before_order_delivered_carrier_date,order_approved_at_before_order_estimated_delivery_date,order_approved_at_before_order_delivered_customer_date,order_delivered_carrier_date_before_order_delivered_customer_date,order_id
15,False,False,False,False,True,False,False,False,dcb36b511fcac050b97cd5c05de84dc3
64,False,False,False,False,True,False,False,False,688052146432ef8253587b930b01a06d
210,False,False,False,False,True,False,False,False,412fccb2b44a99b36714bca3fef8ad7b
415,False,False,False,False,True,False,False,False,56a4ac10a4a8f2ba7693523bb439eede
481,False,False,False,False,True,False,False,False,32e4fa9bb468884309b58b9348de70c3
...,...,...,...,...,...,...,...,...,...
99086,False,False,False,False,True,False,False,False,bb8871a92af73ddadeac0531a0787131
99091,False,False,False,False,True,False,False,False,240ead1a7284667e0ec71d01f80e4d5e
99266,False,False,False,False,True,False,False,False,76a948cd55bf22799753720d4545dd2d
99377,False,False,False,False,True,False,False,False,a6bd1f93b7ff72cc348ca07f38ec4bee


In [ ]:
# 1. Get the order_ids from anomaly case
anomaly_order_ids = df_first_case['order_id'].unique()

# 2. Use to subset
df_first_case_orders = df_orders_delivered[df_orders_delivered['order_id'].isin(anomaly_order_ids)].copy()

display(df_first_case_orders)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
15,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04
64,688052146432ef8253587b930b01a06d,81e08b08e5ed4472008030d70327c71f,delivered,2018-04-22 08:48:13,2018-04-24 18:25:22,2018-04-23 19:19:14,2018-04-24 19:31:58,2018-05-15
210,412fccb2b44a99b36714bca3fef8ad7b,c6865c523687cb3f235aa599afef1710,delivered,2018-07-22 22:30:05,2018-07-23 12:31:53,2018-07-23 12:24:00,2018-07-24 19:26:42,2018-07-31
415,56a4ac10a4a8f2ba7693523bb439eede,78438ba6ace7d2cb023dbbc81b083562,delivered,2018-07-22 13:04:47,2018-07-27 23:31:09,2018-07-24 14:03:00,2018-07-28 00:05:39,2018-08-06
481,32e4fa9bb468884309b58b9348de70c3,e54367d4b00c5cb76d2dfe71b9bdb89c,delivered,2018-07-04 16:49:21,2018-07-05 16:33:06,2018-07-05 14:50:00,2018-07-07 14:41:18,2018-07-23
...,...,...,...,...,...,...,...,...
99086,bb8871a92af73ddadeac0531a0787131,ce757220d9f01ba4887f6bf76ced6a1c,delivered,2018-07-31 09:16:57,2018-08-01 16:31:37,2018-08-01 15:03:00,2018-08-08 01:03:27,2018-08-22
99091,240ead1a7284667e0ec71d01f80e4d5e,fcdd7556401aaa1c980f8b67a69f95dc,delivered,2018-07-02 16:30:02,2018-07-05 16:17:59,2018-07-05 14:11:00,2018-07-10 23:21:47,2018-07-24
99266,76a948cd55bf22799753720d4545dd2d,3f20a07b28aa252d0502fe7f7eb030a9,delivered,2018-01-30 02:41:30,2018-02-04 23:31:46,2018-01-31 18:11:58,2018-03-18 20:08:50,2018-03-02
99377,a6bd1f93b7ff72cc348ca07f38ec4bee,6d63fa86bd2f62908ad328325799152f,delivered,2018-04-20 17:28:40,2018-04-24 19:26:10,2018-04-23 17:18:40,2018-04-28 17:38:42,2018-05-15


In [75]:
# Calculate the time difference between order approval and purchase
df_orders_delivered['approval_time_minutes'] = (
    df_orders_delivered['order_approved_at'] - 
    df_orders_delivered['order_purchase_timestamp']
).dt.total_seconds() / 60  # Convert to minutes

# Get descriptive statistics
approval_time_stats = df_orders_delivered['approval_time_minutes'].describe()

# Display additional percentiles for a more comprehensive view
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
approval_time_percentiles = df_orders_delivered['approval_time_minutes'].quantile(percentiles)

# Display the results
print("Descriptive Statistics for Order Approval Time (in minutes):")
display(approval_time_stats)

print("\nPercentiles of Order Approval Time (in minutes):")
for p, val in zip(percentiles, approval_time_percentiles):
    print(f"{int(p*100)}th percentile: {val:.2f} minutes")

Descriptive Statistics for Order Approval Time (in minutes):


count    96456.000000
mean       616.655887
std       1232.164052
min          0.000000
25%         12.916667
50%         20.600000
75%        871.020833
max      44486.616667
Name: approval_time_minutes, dtype: float64


Percentiles of Order Approval Time (in minutes):
1th percentile: 0.00 minutes
5th percentile: 8.65 minutes
10th percentile: 10.10 minutes
25th percentile: 12.92 minutes
50th percentile: 20.60 minutes
75th percentile: 871.02 minutes
90th percentile: 2075.68 minutes
95th percentile: 2895.79 minutes
99th percentile: 5387.31 minutes


In [76]:
# Calculate the time difference between order approval and purchase
df_first_case_orders['approval_time_minutes'] = (
    df_first_case_orders['order_approved_at'] - 
    df_first_case_orders['order_purchase_timestamp']
).dt.total_seconds() / 60  # Convert to minutes

# Get descriptive statistics
approval_time_stats = df_first_case_orders['approval_time_minutes'].describe()

# Display additional percentiles for a more comprehensive view
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
approval_time_percentiles = df_first_case_orders['approval_time_minutes'].quantile(percentiles)

# Display the results
print("Descriptive Statistics for Order Approval Time (in minutes):")
display(approval_time_stats)

print("\nPercentiles of Order Approval Time (in minutes):")
for p, val in zip(percentiles, approval_time_percentiles):
    print(f"{int(p*100)}th percentile: {val:.2f} minutes")

Descriptive Statistics for Order Approval Time (in minutes):


count     1124.000000
mean      3451.872138
std       2481.925893
min         12.916667
25%       1585.516667
50%       2928.400000
75%       4752.095833
max      17477.483333
Name: approval_time_minutes, dtype: float64


Percentiles of Order Approval Time (in minutes):
1th percentile: 18.75 minutes
5th percentile: 90.93 minutes
10th percentile: 548.26 minutes
25th percentile: 1585.52 minutes
50th percentile: 2928.40 minutes
75th percentile: 4752.10 minutes
90th percentile: 7347.11 minutes
95th percentile: 7766.90 minutes
99th percentile: 9067.18 minutes


For the first anomaly case, it seems like the approval date are not normal. Normally, Olist gets to approve an order in 20-30 minutes. However, orders belonging to this anomaly mostly take > 90 minutes to approve, with half of them needing 3000 minutes (50 hours!) to get approved. This signals either data entry errors or system errors that lead to a late approval datetime entry, even though the order is supposedly already approved and notified to the seller.

So, this first anomaly case does not call for any removal, and we will keep these rows.

#### Second Case:

- Carrier pickup before approval: order_delivered_carrier_date < order_approved_at. This means shipping was marked before the order was even approved. That’s not possible, so it’s a moderate data quality issue.
- Customer received before carrier pickup: order_delivered_customer_date < order_delivered_carrier_date. These are definitely incorrect → a customer can’t receive the order before the courier picks it up.

In [ ]:
# Flag soft anomalies:
df_orders_delivered['carrier_before_approval'] = (
    df_orders_delivered['order_delivered_carrier_date'] < df_orders_delivered['order_approved_at']
)

In [ ]:
# Drop clearly illogical rows:
df_orders_delivered = df_orders_delivered[
    df_orders_delivered['order_delivered_customer_date'] >= df_orders_delivered['order_delivered_carrier_date']
]

In [ ]:
# New features for analysis:
df_corr = df_orders_delivered.copy()

# Approval delay in hours:
df_corr['approval_delay'] = (
    df_corr['order_approved_at'] - df_corr['order_purchase_timestamp']
).dt.total_seconds() / 3600

# Delivery time (carrier → customer) in days:
df_corr['delivery_time_days'] = (
    df_corr['order_delivered_customer_date'] - df_corr['order_delivered_carrier_date']
).dt.days

# Delay vs estimated delivery in days:
df_corr['delay_vs_estimate'] = (
    df_corr['order_delivered_customer_date'] - df_corr['order_estimated_delivery_date']
).dt.days

In [ ]:
# Drop rows with NaN or anomalies in the new features:
corr_cols = ['approval_delay', 'delivery_time_days', 'delay_vs_estimate']
corr_df = df_corr[~df_corr['carrier_before_approval']][corr_cols].dropna()

In [ ]:
# Correlation heatmap for time-based features:
corr_matrix = corr_df.corr()

fig = px.imshow(corr_matrix,
                text_auto=True,
                color_continuous_scale='RdBu_r',
                title='Correlation Heatmap (Time-Based Features Only)')
fig.show()

In [ ]:
# Data with no anomalies (for visualisation):
df_orders_delivered_clean = df_orders_delivered[~df_orders_delivered['carrier_before_approval']].copy()

In [ ]:
# New feature for on-time delivery:
df_orders_delivered_clean['delivered_on_time'] = (
    df_orders_delivered_clean['order_delivered_customer_date'] <= df_orders_delivered_clean['order_estimated_delivery_date']
)

In [ ]:
# On-Time Delivery (OTD) rate calculation:
total_deliveries = len(df_orders_delivered_clean)
on_time_deliveries = df_orders_delivered_clean['delivered_on_time'].sum()

otd_rate = (on_time_deliveries / total_deliveries) * 100
print(f"OTD Rate: {otd_rate:.2f}%")

In [ ]:
# Monthly OTD rate calculation:
df_orders_delivered_clean['delivery_month'] = df_orders_delivered_clean['order_delivered_customer_date'].dt.to_period('M')

monthly_otd = (
    df_orders_delivered_clean.groupby('delivery_month')['delivered_on_time']
    .mean() * 100
).reset_index()

monthly_otd.columns = ['Month', 'OTD (%)']
print(monthly_otd)

In [ ]:
# Filter for deliveries completed in 2017:
df_2017 = df_orders_delivered_clean[
    df_orders_delivered_clean['order_delivered_customer_date'].dt.year == 2017
].copy()

# Calculate total and on-time deliveries:
total_deliveries_2017 = len(df_2017)
on_time_deliveries_2017 = df_2017['delivered_on_time'].sum()

# Calculate OTD:
otd_2017 = (on_time_deliveries_2017 / total_deliveries_2017) * 100
print(f"OTD Rate for 2017: {otd_2017:.2f}%")

In [ ]:
# Filter for deliveries completed in 2018:
df_2018 = df_orders_delivered_clean[
    df_orders_delivered_clean['order_delivered_customer_date'].dt.year == 2018
].copy()

# Calculate total and on-time deliveries:
total_deliveries_2018 = len(df_2018)
on_time_deliveries_2018 = df_2018['delivered_on_time'].sum()

# Calculate OTD:
otd_2018 = (on_time_deliveries_2018 / total_deliveries_2018) * 100
print(f"OTD Rate for 2018: {otd_2018:.2f}%")

In [ ]:
# Monthly OTD rate calculation:
df_orders_delivered_clean['delivery_month'] = df_orders_delivered_clean['order_delivered_customer_date'].dt.to_period('M')

monthly_otd = (
    df_orders_delivered_clean.groupby('delivery_month')['delivered_on_time']
    .mean() * 100
).reset_index()

monthly_otd.columns = ['Month', 'OTD (%)']
monthly_otd['Month'] = monthly_otd['Month'].astype(str)

In [ ]:
# Filter out months with 0% OTD (e.g., 2018-09 and 2018-10):
monthly_otd_filtered = monthly_otd[~monthly_otd['Month'].isin(['2018-09', '2018-10'])].copy()
monthly_otd_filtered = monthly_otd_filtered.sort_values('Month').reset_index(drop=True)

# Prepare values:
y_vals = monthly_otd_filtered['OTD (%)'].values

# Base plot:
fig = go.Figure()

# Main OTD line:
fig.add_trace(go.Scatter(
    x=monthly_otd_filtered['Month'],
    y=y_vals,
    mode='lines+markers',
    name='OTD',
    line=dict(color='blue', width=3),
    marker=dict(color='blue', size=8)
))

# Highlight below 95% points:
below_95 = monthly_otd_filtered[monthly_otd_filtered['OTD (%)'] < 95]

fig.add_trace(go.Scatter(
    x=below_95['Month'],
    y=below_95['OTD (%)'],
    mode='markers+text',
    name='Below 95%',
    marker=dict(color='red', size=10, symbol='circle-open'),
    text=['⚠️' for _ in range(len(below_95))],
    textposition='top center',
    showlegend=True
))

# Final layout:
fig.update_layout(
    title='Monthly On-Time Delivery (OTD) Trend (Excl. Sep-Oct 2018)',
    xaxis_title='Month',
    yaxis_title='OTD (%)',
    yaxis_range=[75, 100],
    template='plotly_white'
)

fig.show()

## Univariate Analysis

In [ ]:
# Calculate delivery time in days:
df_orders_delivered_clean['delivery_time_days'] = (
    df_orders_delivered_clean['order_delivered_customer_date'] - 
    df_orders_delivered_clean['order_delivered_carrier_date']
).dt.days

fig = px.histogram(df_orders_delivered_clean, x='delivery_time_days', nbins=30, 
                   title='Delivery Time (in Days)')
fig.show()

In [ ]:
# Calculate delay vs estimated delivery date:
df_orders_delivered_clean['delay_vs_estimate'] = (
    df_orders_delivered_clean['order_delivered_customer_date'] - df_orders_delivered_clean['order_estimated_delivery_date']
).dt.days

fig = px.histogram(df_orders_delivered_clean, x='delay_vs_estimate', nbins=40, 
                   title='Delivery Delay vs Estimated Date (in Days)')
fig.show()

## Bivariate Analysis

In [ ]:
# Delivery delay vs day of purchase:
df_orders_delivered_clean['purchase_day'] = df_orders_delivered_clean['order_purchase_timestamp'].dt.dayofweek

fig = px.box(df_orders_delivered_clean, x='purchase_day', y='delay_vs_estimate',
             title='Delivery Delay vs Day of Purchase',
             labels={'purchase_day': 'Day of Week (0=Mon)', 'delay_vs_estimate': 'Delay (Days)'})
fig.show()

In [ ]:
# Calculate approval delay:
df_orders_delivered_clean['approval_delay'] = (
    df_orders_delivered_clean['order_approved_at'] - df_orders_delivered_clean['order_purchase_timestamp']
).dt.total_seconds() / 3600  # in hours

fig = px.histogram(df_orders_delivered_clean, x='approval_delay', nbins=40, 
                   title='Approval Delay (Hours)')
fig.show()

## Trivariate Analysis

In [ ]:
# Delivery delay by purchase day:
df_orders_delivered_clean['is_delayed'] = df_orders_delivered_clean['delay_vs_estimate'] > 0

fig = px.box(df_orders_delivered_clean, x='purchase_day', y='delay_vs_estimate',
             color='is_delayed', title='Delay by Purchase Day (Delayed vs Not)')
fig.show()

In [ ]:
df_orders_delivered